<a href="https://colab.research.google.com/github/cbenaventte/CriptoBots/blob/main/crypto_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Below is the combined script that integrates all the functionalities discussed: scraping DexScreener, analyzing tokens, interacting with Rugcheck.xyz, detecting fake volume, and trading on GMGN.ai. I'll also provide step-by-step instructions on how to run the script.

---

### Combined Script

```python
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import logging
from collections import Counter

# Configuration
API_KEY = "your_gmgn_api_key"  # Replace with your GMGN.ai API key
BASE_URL = "https://api.gmgn.ai/v1"  # Replace with the actual GMGN.ai API base URL
DEXSCREENER_URL = "https://dexscreener.com/"  # Replace with the actual DexScreener URL
RUGCHECK_URL = "http://rugcheck.xyz/tokens/"

# Initialize blacklists
TokenBlacklist = set()
DevBlacklist = set()
WalletBlacklist = set()

# Set up logging
logging.basicConfig(level=logging.INFO)

# Load or initialize blacklists
def load_blacklists(filename="blacklists.json"):
    try:
        with open(filename, 'r') as f:
            blacklists = json.load(f)
        return set(blacklists["TokenBlacklist"]), set(blacklists["DevBlacklist"]), set(blacklists["WalletBlacklist"])
    except FileNotFoundError:
        return set(), set(), set()

def save_blacklists(TokenBlacklist, DevBlacklist, WalletBlacklist, filename="blacklists.json"):
    blacklists = {
        "TokenBlacklist": list(TokenBlacklist),
        "DevBlacklist": list(DevBlacklist),
        "WalletBlacklist": list(WalletBlacklist)
    }
    with open(filename, 'w') as f:
        json.dump(blacklists, f)

# Scrape DexScreener
def scrape_dexscreener(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    data = []
    for row in soup.find_all('tr'):
        cols = row.find_all('td')
        if len(cols) > 0:
            data.append([col.text.strip() for col in cols])
    df = pd.DataFrame(data)
    return df

# Analyze pumped coins
def analyze_pumped_coins(df):
    df['Price'] = df['Price'].astype(float)
    df['Volume'] = df['Volume'].astype(float)
    df['PriceChange'] = df['Price'].pct_change()
    pumped_coins = df[df['PriceChange'] > 0.5]  # Example threshold
    return pumped_coins

# Check Rugcheck.xyz
def check_rugcheck_xyz(contract_address):
    url = f"{RUGCHECK_URL}{contract_address}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    status = soup.find("div", class_="status").text.strip() if soup.find("div", class_="status") else None
    return status

# Detect fake volume
def detect_fake_volume(df, volume_threshold=0.8, price_change_threshold=0.1):
    volume_percentile = df['Volume'].quantile(volume_threshold)
    price_change_percentile = df['PriceChange'].quantile(price_change_threshold)
    fake_volume_tokens = df[(df['Volume'] >= volume_percentile) & (df['PriceChange'] <= price_change_threshold)]
    return fake_volume_tokens

# Detect repetitive trades
def detect_repetitive_trades(df, trade_size_column='TradeSize', repetition_threshold=0.5):
    repetitive_tokens = []
    for token, group in df.groupby('Token'):
        trade_sizes = group[trade_size_column].tolist()
        most_common_size, count = Counter(trade_sizes).most_common(1)[0]
        if count / len(trade_sizes) > repetition_threshold:
            repetitive_tokens.append(token)
    return repetitive_tokens

# Detect low liquidity and high volume
def detect_low_liquidity_high_volume(df, liquidity_threshold=0.2, volume_threshold=0.8):
    liquidity_percentile = df['Liquidity'].quantile(liquidity_threshold)
    volume_percentile = df['Volume'].quantile(volume_threshold)
    suspicious_tokens = df[(df['Liquidity'] <= liquidity_percentile) & (df['Volume'] >= volume_percentile)]
    return suspicious_tokens

# Filter tokens based on Rugcheck and bundled supply
def filter_tokens(df, TokenBlacklist, DevBlacklist):
    for index, row in df.iterrows():
        contract_address = row['ContractAddress']
        status = check_rugcheck_xyz(contract_address)
        if status != "Good":
            TokenBlacklist.add(row['Token'])
            continue
        if check_supply_distribution(contract_address):
            TokenBlacklist.add(row['Token'])
            DevBlacklist.add(row['Creator'])
    return TokenBlacklist, DevBlacklist

# Check supply distribution
def check_supply_distribution(contract_address):
    # Placeholder: Use an API like Etherscan to fetch token holder data
    return False  # Replace with actual logic

# Exclude blacklisted tokens
def exclude_blacklisted_tokens(df, TokenBlacklist, DevBlacklist):
    df = df[~df['Token'].isin(TokenBlacklist)]
    df = df[~df['Creator'].isin(DevBlacklist)]
    return df

# GMGN.ai API functions
def transfer_tokens(from_wallet, to_wallet, token_address, amount):
    url = f"{BASE_URL}/transfer"
    payload = {
        "from_wallet": from_wallet,
        "to_wallet": to_wallet,
        "token_address": token_address,
        "amount": amount
    }
    response = requests.post(url, headers=headers, json=payload)
    return response.json()

def purchase_tokens(wallet_address, token_address, amount, price):
    url = f"{BASE_URL}/purchase"
    payload = {
        "wallet_address": wallet_address,
        "token_address": token_address,
        "amount": amount,
        "price": price
    }
    response = requests.post(url, headers=headers, json=payload)
    return response.json()

def check_token_balance(wallet_address, token_address):
    url = f"{BASE_URL}/balance"
    params = {
        "wallet_address": wallet_address,
        "token_address": token_address
    }
    response = requests.get(url, headers=headers, params=params)
    return response.json()

# Main workflow
def main():
    global TokenBlacklist, DevBlacklist, WalletBlacklist
    TokenBlacklist, DevBlacklist, WalletBlacklist = load_blacklists()

    # Scrape and analyze data
    df = scrape_dexscreener(DEXSCREENER_URL)
    df = analyze_pumped_coins(df)

    # Detect fake volume and suspicious tokens
    fake_volume_tokens = detect_fake_volume(df)
    repetitive_tokens = detect_repetitive_trades(df)
    suspicious_tokens = detect_low_liquidity_high_volume(df)

    # Filter tokens based on Rugcheck and bundled supply
    TokenBlacklist, DevBlacklist = filter_tokens(df, TokenBlacklist, DevBlacklist)

    # Exclude blacklisted tokens
    cleaned_df = exclude_blacklisted_tokens(df, TokenBlacklist, DevBlacklist)

    # Save updated blacklists
    save_blacklists(TokenBlacklist, DevBlacklist, WalletBlacklist)

    # Output cleaned data
    print("Cleaned Data (excluding blacklisted tokens and developers):")
    print(cleaned_df)

    # Example: Automate trading
    wallet_address = "0xYourWalletAddress"
    token_address = "0xTokenContractAddress"
    amount = 100
    price = 0.01

    purchase_response = purchase_tokens(wallet_address, token_address, amount, price)
    print("Purchase Response:", purchase_response)

    to_wallet = "0xRecipientWalletAddress"
    transfer_response = transfer_tokens(wallet_address, to_wallet, token_address, amount)
    print("Transfer Response:", transfer_response)

if __name__ == "__main__":
    main()
```

---

### Instructions to Run the Script

1. **Install Required Libraries**:
   Ensure you have the required Python libraries installed:
   ```bash
   pip install requests beautifulsoup4 pandas
   ```

2. **Set Up Configuration**:
   - Replace `your_gmgn_api_key` with your actual GMGN.ai API key.
   - Update `BASE_URL` with the correct GMGN.ai API endpoint.
   - Update `DEXSCREENER_URL` and `RUGCHECK_URL` if necessary.

3. **Run the Script**:
   Save the script to a file, e.g., `crypto_bot.py`, and run it:
   ```bash
   python crypto_bot.py
   ```

4. **Review Output**:
   - The script will output cleaned data (excluding blacklisted tokens and developers).
   - It will also print responses for token purchases and transfers.

5. **Monitor and Debug**:
   - Check the logs for any errors or issues.
   - Adjust thresholds and parameters as needed.

---

### Notes:
- **API Key Security**: Never hardcode your API key in the script. Use environment variables or a secure vault.
- **Error Handling**: Add more robust error handling for production use.
- **Testing**: Test the script in a sandbox environment before using it with real funds.

Let me know if you need further assistance!